<a href="https://colab.research.google.com/github/Drausio72/Bolsa/blob/main/Bolsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Carregar Dados
ticker = "EGIE3.SA" # @param {type:"string"}
start_date = "2019-05-15" # @param {type:"date"}
end_date = "2024-05-15" # @param {type:"date"}
import numpy as np
import pandas as pd
import yfinance as yf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Carregar dados
def load_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Close'].values.reshape(-1, 1)  # Focando nos preços de fechamento

In [ ]:
# @title Preparar dados para o LSTM
def prepare_data(data, look_back=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    X, Y = [], []
    for i in range(look_back, len(scaled_data)):
        X.append(scaled_data[i-look_back:i, 0])
        Y.append(scaled_data[i, 0])
    X, Y = np.array(X), np.array(Y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, Y, scaler


In [ ]:
# @title Construir e treinar o modelo LSTM
def build_and_train_model(X, Y):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, Y, epochs=1000, batch_size=32)
    return model



In [ ]:
# @title Fazer previsões e calcular variação percentual
def predict_and_calculate_changes(model, data, scaler):
    last_batch = data[-1]  # Último lote de dados
    next_day_prediction = model.predict(np.array([last_batch]))
    next_day_prediction = scaler.inverse_transform(next_day_prediction)  # Desnormalizar
    last_price = scaler.inverse_transform([data[-1][-1]])  # Último preço conhecido
    change_percent = ((next_day_prediction - last_price) / last_price) * 100
    return change_percent[0][0]



In [ ]:
# @title Inicialização e execução
data = load_data(ticker, start_date, end_date)
X, Y, scaler = prepare_data(data)
model = build_and_train_model(X, Y)
change_percent = predict_and_calculate_changes(model, X, scaler)
print(f"Variação percentual para o próximo dia de trading: {change_percent:.2f}%")

[*********************100%%**********************]  1 of 1 completed


Epoch 1/1000
37/37 [==============================] - 3s 7ms/step - loss: 0.0197
Epoch 2/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0058
Epoch 3/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0046
Epoch 4/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0040
Epoch 5/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0036
Epoch 6/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0033
Epoch 7/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0030
Epoch 8/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0029
Epoch 9/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0027
Epoch 10/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0026
Epoch 11/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0025
Epoch 12/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.0023
Epoch 13/1000
37/37 [====